In [2]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from langchain_community.embeddings import SentenceTransformerEmbeddings

In [3]:
import os
import qdrant_client
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()

False

In [4]:
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
client = QdrantClient(":memory:")

client.create_collection(
    collection_name="rag",
    vectors_config=VectorParams(size=384, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="rag",
    embedding=embeddings,
)

print("✅ Qdrant on-disk setup completed!")


C:\Users\mayur\AppData\Local\Temp\ipykernel_12900\381066254.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
C:\Users\mayur\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Qdrant on-disk setup completed!


In [5]:

chunks = PyPDFLoader("./knowledge/ML.pdf").load_and_split()

In [6]:
from langchain_core.documents import Document
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(chunks))]
vector_store.add_documents(documents=chunks, ids=uuids)

print("✅ Documents added to Qdrant!")


✅ Documents added to Qdrant!


In [7]:
query = "What is Machinelearning?"
results = vector_store.similarity_search(query, k=2)

print("\n🔍 Search Results:")
for res in results:
    print(f"📄 {res.page_content} [{res.metadata}]")



🔍 Search Results:
📄 1. Introduction to Machine Learning 
1.1 Definition and importance of ML : Machine learning is a branch of artificial 
intelligence (AI) and computer science which focuses on the use of data and algorithms 
to imitate the way that humans learn, gradually improving its accuracy. 
1.2 Types of machine learning:  
a) Supervised Learning 
Supervised learning is a type of machine learning where the algorithm is trained on labeled 
data. This means that each training example is paired with an output label. The goal is for the 
model to learn to map inputs to the correct outputs. 
• Common Algorithms: Linear Regression, Logistic Regression, Decision Trees, 
Random Forests, Support Vector Machines (SVM), K-Nearest Neighbors (KNN), 
Neural Networks. 
 
b) Unsupervised Learning 
Unsupervised learning is a type of machine learning where the algorithm is trained on 
unlabeled data. The goal is to identify patterns and structures within the data. 
• Common Algorithms: K-Means C

In [16]:
def response_generator(vectordb, query, llm):
    context = vectordb.similarity_search(query, k=2)

    # for res in results:
    #     print(f"📄 {res.page_content} [{res.metadata}]")
    template = f"""
    You are an intelligent assistant designed to provide accurate and concise answers based on the context provided. 
    Follow these rules strictly:
    1. Use ONLY the information provided in the context to answer the question.
    2. If the context does not contain enough information to answer the question, say "I don't know."
    3. Do not make up or assume any information outside of the context.
    4. Keep your answer concise and to the point (maximum 3 sentences).

    Context:
    {context}

    Question:
    {query}

    Helpful Answer:
    """
    ans = llm.invoke(template)
    return ans.content

In [17]:
from langchain_ollama import ChatOllama
llm = ChatOllama(
    model = "mistral",
    temperature = 0,
    num_predict = 256,
    # other params ...
)

In [18]:
response_generator(vector_store, "What is Machinelearning?", llm)

' Machine learning is a branch of artificial intelligence that focuses on using data and algorithms to imitate human learning, gradually improving its accuracy.'